# Verificando conteúdo do dataset de treinamento
Vamos verificar se o conteúdo dos tweets capturados realmente corresponde ao sentimento
rotulado e excluir os tweets que não tem nada a ver com o sentimento. Para isso, vamos
verificar as palavras e temas mais frequentes, através das colocações, distribuição de
frequência, trigramas e nuvem de palavras.

In [1]:
import pandas as pd
import nltk

path = '../datasets/'
arquivo = path + 'tweets_ekman.csv'

df_ekman = pd.read_csv(arquivo)
df_ekman.head()

,Unnamed: 0,Texto,Sentimento
0,0,"quero desejar a todos amigos um , e creia 2...",feliz
1,1,mais um ano ao lado dessas pessoas maravilhosa...,feliz
2,2,"final de tarde perfeito, estou renovada e pron...",feliz
3,3,vamos adorar só aquele que é digno de todo o l...,feliz
4,4,jesus! esse é meu sentimento neste dia. pas...,feliz


## Verificando colocações
Colocações são os bigramas que mais se repetem e podem nos dar alguma
informação sobre o que ée mais relevante em um corpus. Neste caso, os
tweets relacionados a cada uma das 5 emoções de Ekman.

In [2]:
def construir_corpus(serie_texto):
    textos = list(serie_texto)
    corpus = '. '.join(textos)
    tokens = nltk.word_tokenize(corpus)
    corpus = nltk.Text(tokens)

    return corpus

def escrever_colocacoes(corpus, nome_arquivo):
    colocacoes = corpus.collocation_list()
    path = 'arquivos/' + nome_arquivo
    colocacoes_writer = open(path, 'w', encoding='utf-8')

    for colocacao in colocacoes:
        colocacoes_writer.write(colocacao + '\n')

    print('Arquivo criado com êxito!')

## Colocações em trigramas
Anteriormente, verificamos as colocações de duas palavras. Ou seja,
bigramas. Vamos verificar agora quais os trigramas mais frequentes

In [12]:
"""
Implementação da sumarização extrativa por tf-idf utilizando NLTK, de acordo com
o tutorial disponível em: https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3
"""
import math
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


def tokenizar(texto):
    """
    Função para tokenizar as sentenças do texto passado como argumento.

    :param texto: texto original
    :return: o texto tokenizado
    """
    periodos = sent_tokenize(texto)
    total_de_documentos = len(periodos)

    return total_de_documentos


def matriz_frequencia(periodos):
    """
    Cria uma matriz de frequência das palavras em cada sentença. Cada sentença é a chave e o valor é um
    dicionário com a frequência de palavras.

    :param periodos: A sentença que será analisada
    :return: Uma matriz de frequência das palavras por sentença
    """
    matriz_frequencia = {}
    stopWords = set(stopwords.words('portuguese'))
    ps = PorterStemmer()

    for periodo in periodos:
        tabela_freq = {}
        palavras = word_tokenize(periodo)
        for palavra in palavras:
            palavra = palavra.lower()
            palavra = ps.stem(palavra)
            if palavra in stopWords:
                continue
            if palavra in tabela_freq:
                tabela_freq[palavra] += 1
            else:
                tabela_freq[palavra] = 1

        matriz_frequencia[periodo[:15]] = tabela_freq

    return matriz_frequencia


def criar_matriz_tf(matriz_frequencia):
    """
    Calcula a frequência de termo de cada palavra e gera uma matriz.
    Aqui cada parágrafo é considerado como um documento e o termo como uma palavra no parágrafo

    :param matriz_frequencia: Matriz de frequência dos termos (palavras) por documento (parágrafo)
    :return: Matriz TF (Term Frequency)
    """
    matriz_tf = {}

    for periodo, tabela_frequencia in matriz_frequencia.items():
        tabela_tf = {}

        quantidade_palavras_periodo = len(tabela_frequencia)
        for palavra, cont in tabela_frequencia.items():
            tabela_tf[palavra] = cont / quantidade_palavras_periodo

        matriz_tf[periodo] = tabela_tf

    return matriz_tf


def criar_documentos_por_palavras(matriz_freq):
    """
    Cria uma matriz que contém quantas sentenças possuem determinada palavra.

    :param matriz_freq: Matriz de frequência das palavras por parágrafo
    :return: Matriz de quantas sentenças possuem determinada palavra.
    """
    tabela_palavra_por_doc = {}

    for periodo, tabela_frequencia in matriz_freq.items():
        for palavra, cont in tabela_frequencia.items():
            if palavra in tabela_palavra_por_doc:
                tabela_palavra_por_doc[palavra] += 1
            else:
                tabela_palavra_por_doc[palavra] = 1

    return tabela_palavra_por_doc


def criar_matriz_idf(matriz_frequencia, n_docs_por_palavras, total_docs):
    """
    Calcula a matriz idf, considerando o parágrafo como o documento
    e cada palavra no parágrafo como termo.

    :param matriz_frequencia: Matriz de frequência das palavras por parágrafo
    :param n_docs_por_palavras: Quantas sentenças possuem cada palavra
    :param total_docs: Quantidade de documentos (parágrafos)
    :return: Matriz IDF (Inverse Document Frequency)
    """
    matriz_idf = {}

    for periodo, tabela_frequencia in matriz_frequencia.items():
        tabela_idf = {}
        for palavra in tabela_frequencia.keys():
            tabela_idf[palavra] = math.log10(total_docs / float(n_docs_por_palavras[palavra]))
        matriz_idf[periodo] = tabela_idf

    return matriz_idf


def criar_matriz_tfidf(matriz_tf, matriz_idf):
    """
    Multiplica os termos da matriz tf e da idf e cria uma nova matriz com o resultado

    :param matriz_tf: Matriz TF (Term Frequency)
    :param matriz_idf: Matriz IDF (Inverse Document Frequency)
    :return: Matriz TF-IDF
    """
    matriz_tfidf = {}

    for (periodo1, tabela_freq1), (periodo2, tabela_freq2) in zip(matriz_tf.items(), matriz_idf.items()):
        tabela_tfidf = {}
        for (palavra1, valor1), (palavra2, valor2) in zip(tabela_freq1.items(),
                                                          tabela_freq2.items()):
            tabela_tfidf[palavra1] = float(valor1 * valor2)
        matriz_tfidf[periodo1] = tabela_tfidf

    return matriz_tfidf


def pontuar_periodos(matriz_tfidf) -> dict:
    """
    Dá um peso para cada parágrafo de acordo com sua pontuação na matriz TF-IDF

    :param matriz_tfidf: Matriz TF-IDF
    :return: Matriz com pesos para cada sentença
    """
    valorPeriodo = {}

    for periodo, tabela_freq in matriz_tfidf.items():
        pontuacao_total_por_periodo = 0
        cont_palavras_no_periodo = len(tabela_freq)
        for palavra, pontuacao in tabela_freq.items():
            pontuacao_total_por_periodo += pontuacao

        valorPeriodo[periodo] = pontuacao_total_por_periodo / cont_palavras_no_periodo

    return valorPeriodo


def encontrar_pontuacao_media(valor_periodo):
    """
    Calcula a pontuação média das pontuções dadas para cada sentença

    :param valor_periodo: Pontuações dadas às sentenças
    :return: Pontuação média entre as sentenças
    """
    valores_soma = 0

    for entrada in valor_periodo:
        valores_soma += valor_periodo[entrada]

    media = (valores_soma / len(valor_periodo))

    return media


def gerar_resumo(periodos, valor_periodo, pont_media):
    """
    Gera um resumo com base na pontuação média das sentenças. Serão colocadas no resumo apenas as sentenças que
    possuírem um valor superior ao produto da limiar escolhida pela pontuação média as sentenças.

    :param periodos: texto original tokenizado
    :param valor_periodo: Pontuação de uma sentença específica
    :param pont_media: Pontuação média entre todas as sentenças
    :return: Texto resumido
    """
    contador_periodo = 0
    resumo = ''

    for periodo in periodos:
        if periodo[:15] in valor_periodo and valor_periodo[periodo[:15]] >= pont_media:
            resumo += " " + periodo
            contador_periodo += 1

    return resumo


def tf_idf(texto):
    """
    Faz o resumo de um texto utilizando o algoritmo TF-IDF

    :param texto: O texto original a ser resumido
    :return: O texto resumido
    """

    # Tokenizamos as sentenças, ao invés das palavras
    periodos = sent_tokenize(texto)

    total_docs = len(periodos)

    # Criamos uma matriz de frequência das palavras em cada sentença
    # Cada sentença é a chave e o valor é um dicionário com a frequência de palavras
    matriz_freq = matriz_frequencia(periodos)

    # Calculamos a frequência de termo de cada palavra e geramos uma matriz
    # Consideramos cada parágrafo como um documento e o termo como uma palavra no parágrafo
    matriz_tf = criar_matriz_tf(matriz_freq)

    # Uma matriz que contém quantas sentenças possuem determinada palavra
    cont_doc_por_palavras = criar_documentos_por_palavras(matriz_freq)

    # Aqui finalmente calculamos a matriz idf, lembrando que consideramos o parágrafo como o documento
    # e cada palavra no parágrafo como termo
    matriz_idf = criar_matriz_idf(matriz_freq, cont_doc_por_palavras, total_docs)

    # Agora multiplicamos os termos da matriz tf e da idf e criamos uma nova matriz com o resultado
    matriz_tfidf = criar_matriz_tfidf(matriz_tf, matriz_idf)

    # Damos um peso para cada parágrafo de acordo com sua pontuação na matriz TF-IDF
    pontuacao_periodos = pontuar_periodos(matriz_tfidf)

    # Cada algoritmo de sumarização utiliza uma forma diferente para calcular a limiar
    # Aqui, calculamos a pontuação média das pontuções dadas para cada sentença
    pontuacao_media = encontrar_pontuacao_media(pontuacao_periodos)

    # Por fim, são colocadas no resumo apenas as frases que pssuem uma pontuação maior que
    # o valor rescolhido como limiar. Aqui, vamos escolher uma limiar de 1.1
    limiar = 1.1
    resumo = gerar_resumo(periodos, pontuacao_periodos, limiar * pontuacao_media)

    return resumo

def escrever_resumo(texto, nome_arquivo):
    resumo = tf_idf(texto)
    path = 'arquivos/' + nome_arquivo

    with open(path, 'w', encoding='utf-8') as escritor:
        escritor.writelines(resumo)

    print('Resumo criado com êxito!')

In [13]:
# ['feliz', 'medo', 'triste', 'raiva', 'nojo']
sentimentos = list(df_ekman['Sentimento'].unique())

for sentimento in sentimentos:
    txt = df_ekman.loc[df_ekman['Sentimento'] == sentimento, 'Texto']
    corpus = construir_corpus(txt)

    arquivo = sentimento + '_colocacoes.txt'
    escrever_colocacoes(corpus=corpus, nome_arquivo=arquivo)
    nome_resumo = sentimento + '_resumo.txt'
    texto = ' '.join(corpus.tokens)
    escrever_resumo(texto, nome_resumo)

Arquivo criado com êxito!
Resumo criado com êxito!
Arquivo criado com êxito!
Resumo criado com êxito!
Arquivo criado com êxito!
Resumo criado com êxito!
Arquivo criado com êxito!
Resumo criado com êxito!
Arquivo criado com êxito!
Resumo criado com êxito!
